In [ ]:
import numpy as np
import scipy.integrate as si
import matplotlib.pyplot as plt

In [ ]:
def fun(x, y):
    """
    Specify the dynanimcal equation

    Parameters
    ----------
    x : np.ndarray
        Shape (m,) array specifying mesh points
    y : np.ndarray
        Shape (n, m) array holding y values

    Returns
    -------
    dy/dx : np.ndarray
        Shape (n,) array containing RHS of dynamical equations evaluated at x.
    """
    y_1_prime = y[1]
    y_2_prime = -np.exp(y[0])
    return np.vstack((y_1_prime, y_2_prime))

In [ ]:
def bc(ya, yb):
    """
    Enforce the boundary conditions.

    Parameters
    ----------
    ya : np.ndarray
        Shape (n,) array specifying y values at x=a
    yb : np.ndarray
        Shape (n,) array specifying y values at x=b

    Returns
    -------
    res : np.ndarray
        Shape (n,) array evaluating BC residuals for y.

    """
    res1 = ya[0] - 1
    res2 = yb[0] + 1
    res = np.array([res1, res2])
    return res

In [ ]:
mesh_points = 50
x = np.linspace(-1, 1, mesh_points)
y = np.zeros((2, mesh_points))
y[0, 0] = 1
y[0, -1] = -1

In [ ]:
sol = si.solve_bvp(fun, bc, x, y)

In [ ]:
plt.plot(sol.x, sol.y[0])
plt.xlabel("x")
plt.ylabel("y")
plt.title("Solved bvp")
plt.show()

Now solving an example with parameters

In [ ]:
def fun(x, y, p):
    k = p[0]
    y_1_prime = y[1]
    y_2_prime = -(k ** 2) * y[0]
    y_prime = np.vstack((y_1_prime, y_2_prime))
    return y_prime

In [ ]:
def bc(ya, yb, p):
    k = p[0]
    res_1 = ya[0] - 1
    res_2 = yb[0] - 1
    res_3 = ya[1] - k
    return np.array([res_1, res_2, res_3])

In [ ]:
mesh_points = 50
k = 6
x = np.linspace(-1, 1, mesh_points)
y = np.zeros((2, mesh_points))
y[0, 0] = 1
y[0, -1] = 1
y[1, 0] = k

In [ ]:
sol = si.solve_bvp(fun, bc, x, y, p=[k])

In [ ]:
plt.plot(sol.x, sol.y[0])
plt.xlabel("x")
plt.ylabel("y")
plt.title("Solved bvp")
plt.show()

In [ ]:
sol.p

## Next we solve my problem

![Equations](eqn_pic.png)

For now set $B = \rho = 1$ and $g = 10$ with $m^a = 0$

In [ ]:
def fun(x, y, p):
    """
    First we will solve with zero active moment.
    """
    l = p[0]
    eq1 = np.cos(y[2]) * l
    eq2 = np.sin(y[2]) * l
    eq3 = y[3]
    eq4 = 10 * l ** 3 * np.cos(y[2])
    eqn_arr = np.vstack((eq1, eq2, eq3, eq4))
    return eqn_arr

In [ ]:
def bc(ya, yb, p):
    """
    Enforce all the boundary residuals.
    """
    l = p[0]
    res_1 = ya[2]
    res_2 = ya[3]
    res_3 = yb[0]
    res_4 = yb[1]
    res_5 = yb[2]
    return np.array([res_1, res_2, res_3, res_4, res_5])

In [ ]:
mesh_points = 50
l = 3
x = np.linspace(0, 1, mesh_points)
y = np.zeros((4, mesh_points))
y[0, 0] = 2
y[0, -1] = 0
y[1, -1] = 0
y[1, 0] = 2
y[2, 0] = 0
y[2, -1] = 0
y[3, 0] = 0

In [ ]:
y.shape

In [ ]:
sol = si.solve_bvp(fun, bc, x, y, p=[l])

In [ ]:
sol.status

In [ ]:
sol.message

In [ ]:
plt.plot(sol.x, sol.y[0], label="x")
plt.plot(sol.x, sol.y[1], label="y")
plt.plot(sol.x, sol.y[2], label=r"$\theta$")
plt.plot(sol.x, sol.y[3], label=r"$\frac{d\theta}{dr}$")
plt.legend()
plt.show()

In [ ]:
sol.p

## Didn't work. Let me do a simplified version just solving for the $\theta s$

So I will solve
\begin{equation*}
    \frac{d\theta_1}{dr} = \theta_2 \\
    \frac{d\theta_2}{dr} = - \frac{l}{B}\frac{dm^a(r)}{dr} + \frac{\rho g r l^3}{B} cos\theta_1 \\
    \theta_1(0) = 0 \\ 
    \theta_1(1) = 0 \\ 
    \theta_2(0) = -l m^a(0)
\end{equation*}

I'll choose something simple where $m^a(r) = r + 1$

In [ ]:
def fun(x, y, p):
    l = p[0]
    t_1_prime = y[1]
    t_2_prime = -l + l ** 3 * np.cos(y[0])
    return np.vstack((t_1_prime, t_2_prime))


def bc(ya, yb, p):
    l = p[0]
    res1 = ya[0]
    res2 = ya[1] + l
    res3 = yb[0]
    return np.array([res1, res2, res3])


mesh_points = 50
l = 3
x = np.linspace(0, 1, mesh_points)
y = np.zeros((2, mesh_points))
y[0, 0] = 0
y[0, -1] = 0
y[1, -1] = -l

In [ ]:
sol = si.solve_bvp(fun, bc, x, y, p=[l])
plt.plot(sol.x, sol.y[0], label=r"$\theta$")
plt.plot(sol.x, sol.y[1], label=r"$\frac{d\theta}{dr}$")
plt.legend()
plt.show()

In [ ]:
x = sol.p[0] * si.cumtrapz(np.cos(sol.y[0]), sol.x)
x -= x[-1]
y = sol.p[0] * si.cumtrapz(np.sin(sol.y[0]), sol.x)
y -= y[-1]
plt.plot(x, y)
plt.show()

In [ ]:
def cost(sol, m_a, alpha=0.5):
    """
    Compute the cost function we are trying to minimize.

    Note we have the computed values as a function of r so we should
    integrate from 0 to 1 instead of 0 to l.
    """
    r = sol.x
    dtheta_dr = sol.y[1]
    active_moment = m_a(r)
    work = 0.5 * si.trapz(active_moment * dtheta_dr, r)
    y = sol.p[0] * si.cumtrapz(np.sin(sol.y[0]), sol.x)
    y -= y[-1]
    height = y[0]
    cost = (1 - alpha) * work - alpha * height
    return cost


def m_a_lin(r):
    return r + 1

In [ ]:
cost(sol, m_a_lin)

In [ ]:
type(sol)

Slightly more complex $m^a = r^2 + .5$

In [ ]:
def fun(x, y, p):
    l = p[0]
    t_1_prime = y[1]
    t_2_prime = -l * 2 * x + l ** 3 * x * np.cos(y[0])
    return np.vstack((t_1_prime, t_2_prime))


def bc(ya, yb, p):
    l = p[0]
    res1 = ya[0]
    res2 = ya[1] + l * 0.5
    res3 = yb[0]
    return np.array([res1, res2, res3])


mesh_points = 50
l = 3
x = np.linspace(0, 1, mesh_points)
y = np.zeros((2, mesh_points))
y[0, 0] = 0
y[0, -1] = 0
y[1, 0] = -l * 0.5

In [ ]:
sol = si.solve_bvp(fun, bc, x, y, p=[l])
plt.plot(sol.x, sol.y[0], label=r"$\theta$")
plt.plot(sol.x, sol.y[1], label=r"$\frac{d\theta}{dr}$")
plt.xlabel("r")
plt.legend()
plt.show()

In [ ]:
x = sol.p[0] * si.cumtrapz(np.cos(sol.y[0]), sol.x)
x -= x[-1]
y = sol.p[0] * si.cumtrapz(np.sin(sol.y[0]), sol.x)
y -= y[-1]
plt.plot(x, y)
plt.show()

In [ ]:
def m_a_sq(r):
    return r ** 2 + 0.5


cost(sol, m_a_sq)